In [ ]:
# Define useful paths
SCRATCH_ROOT = "/storage/vbutoi/scratch/ESE"
CONFIG_ROOT = "/storage/vbutoi/projects/ESE/configs"
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/projects/ESE')

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
%%yaml default_cfg 

experiment:
  seed: 40
  seed_range: 1
  val_first: False 
  torch_compile: False 
  torch_mixed_precision: False
  sys_paths:
    - "/storage/vbutoi/projects"
    - "/storage/vbutoi/libraries"
    - "/storage/vbutoi/projects/ESE"
  data_paths:
    - "/storage"
    - "/storage/vbutoi/datasets"

dataloader:
  batch_size: 16 
  num_workers: 4 
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 1.0e-4 
  
train:
  epochs: 500 
  eval_freq: 5 

log:
  checkpoint_freq: 5 

# Define the data.

In [ ]:
%%yaml base_data_cfg 

# Used for additional data experiment.
data:
  _class: 'ese.datasets.ISLES'
  train_kwargs:
    split: 'train'
  val_kwargs: 
    split: 'val'

In [ ]:
%%yaml finetune_data_cfg 

# Used for additional data experiment.
data:
  use_pt_data_cfg: True
  train_kwargs:
    split: 'train'
  val_kwargs: 
    split: 'val'

# Define the Loss Function config.

In [ ]:
%%yaml base_loss_cfg

loss_func: 
  _class: ese.losses.SoftDiceLoss
  from_logits: True
  batch_reduction: 'mean' 
  ignore_empty_labels: False 

In [ ]:
%%yaml finetune_loss_cfg

loss_func: 
  _class: ese.losses.SoftDiceLoss
  from_logits: True
  batch_reduction: 'mean' 
  ignore_empty_labels: False 

# Define the Model config.

In [ ]:
%%yaml base_model_cfg  

model:
  _class: ese.models.unet.UNet
  filters: [64, 64, 64, 64, 64, 64, 64] 
  convs_per_block: 3 # Good default for UNets.
  dims: 3

In [ ]:
%%yaml finetune_model_cfg  

model:
  _class: '?'
  pretrained_dir: '?'

# Define the Callbacks config.

In [ ]:
%%yaml base_callbacks_cfg

log:
  root: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      from_logits: True

callbacks:
  step:
    - ionpy.callbacks.ShowPredictions:
        vis_type: 'segmentation'
  epoch:
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.WandbLogger:
        entity: 'vbutoi'
        project: 'SemanticCalibration'
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

In [ ]:
%%yaml finetune_callbacks_cfg

log:
  root: '?'
  metrics:
    ece_loss:
      _fn: ese.metrics.image_ece_loss
      num_prob_bins: 15
      from_logits: True
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      from_logits: True

callbacks:
  step:
    - ionpy.callbacks.ShowPredictions:
        vis_type: 'segmentation'
  epoch:
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.WandbLogger:
        entity: 'vbutoi'
        project: 'SemanticCalibration'
    - ionpy.callbacks.ModelCheckpoint:
        monitor: 
          - dice_score
          - ece_loss
        phase: val

# Define the config combos.

In [ ]:
# Training standard segmentation models.
base_cfg = Config(default_cfg).update([
    base_data_cfg,
    base_loss_cfg,
    base_model_cfg, 
    base_callbacks_cfg
])

# Training models on top of base ones.
finetune_cfg = Config(default_cfg).update([
    finetune_data_cfg,
    finetune_loss_cfg,
    finetune_model_cfg, 
    finetune_callbacks_cfg 
])

# Experimental Variations.

In [ ]:
%%yaml experiment_cfg 

group: "ISLES_TS"

# model: 
#     _class: "ese.models.TS"
#     pretrained_dir: "/storage/vbutoi/scratch/ESE/training/older_runs/2024/September_2024/09_25_24_ISLES_3D_Dice_HeavyAug/20240925_234556-HGRQ-56cd3bf7df7a1b7d2a453b296d64407d"

dataloader:
    batch_size: 1
    num_workers: 1

In [ ]:
from ionpy.experiment.generate_configs import get_training_configs

# Get the different experiment cfg yamls.
updated_base_cfg, train_cfgs = get_training_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg,
    # base_cfg=finetune_cfg,
    config_root=CONFIG_ROOT,
    scratch_root=SCRATCH_ROOT,
    add_date=True
)

In [ ]:
len(train_cfgs)

# Running Jobs

In [ ]:
# ####### FOR DEBUGGIN
from ionpy.slite import run_exp
from ese.experiment import CalibrationExperiment

run_exp(
    config=train_cfgs[0], 
    experiment_class=CalibrationExperiment,
    run_name='debug',
    show_examples=True,
    track_wandb=False,
    gpu='0',
)

In [ ]:
%%yaml submit_cfg

mode: "local"
group: "training"
add_date: True
track_wandb: True
scratch_root: "/storage/vbutoi/scratch/ESE"

In [ ]:
# # FOR SUBMISSION
# from ionpy.slite import submit_exps
# from sebench.experiment.train import CalibrationExperiment 

# submit_exps(
#     submit_cfg=submit_cfg,
#     config_list=train_cfgs,
#     exp_cfg=experiment_cfg,
#     base_cfg=updated_base_cfg,
#     experiment_class=CalibrationExperiment,
#     available_gpus=['0', '1', '2', '3', '4', '5', '6', '7'],
# )